# Chapter 1: RISC-V Assembly

[RISC-V](https://riscv.org/) is an [instruction set architecture](https://en.wikipedia.org/wiki/Instruction_set_architecture) designed to be simple and extensible.
These design principles, as well as it being an open standard, made it a popular target for computer science research, and it's slowly picking up steam as a platform for personal computers and accelerators in industry.
As part of my PhD, I am working on writing a compiler for neural networks targeting AI accelerators that extend the RISC-V ISA, with the neural networks and accelerator hardware developed by other partners in [CONVOLVE](https://convolve.eu/).

This is the first of a series of blog posts describing how to write such a compiler in Python, using open-source frameworks such as [riscemu](https://github.com/antonlydike/riscemu) and [xDSL](https://xdsl.dev/).

We'll start with a simple example of a program that prints "Hello, World!" in modified RISC-V assembly:

```
main:
    printf "Hello, World!"
    li  a7, 93
    ecall
```


Let's walk through the above code line by line:

```
main:
```

Text followed by a colon is a label, in this case `main` is the name of our main function
and entry point.

```
printf "Hello World!"
```

`printf` is a custom instruction in `riscemu`, our RISC-V emulator. It lets us easily
inspect values in our registers. Usually, there are many more steps involved in printing
text in real programs on operating systems, which are not very relevant to neural network
compilation, so we'll use this shortcut here and in the rest of the series.

```
li a7, 93
ecall
```

`ecall` is the instruction to call a syscall, and the syscall to call is specified by the
value in `a7`.
When that value is `93`, the program exits.
[Here is a table](https://jborza.com/post/2021-05-11-riscv-linux-syscalls/) of the syscalls and their codes for Linux on RISC-V.

To run the assembly, we'll use a riscemu, a RISC-V emulator written in Python.
Here's how we can use riscemu to execute the assembly above:

In [3]:
from io import StringIO

from riscemu.config import RunConfig
from riscemu.instructions import RV32D, RV32F, RV32I, RV32M, RV_Debug
from riscemu.riscemu_main import RiscemuMain, RiscemuSource

# Initialise a riscemu instance with a default configuration
main = RiscemuMain(RunConfig())
# We'll use the following instruction sets for our emulation
main.selected_ins_sets = [RV32I, RV32M, RV32F, RV32D, RV_Debug]
# Allow assembly files to be passed to main
main.register_all_program_loaders()

code = """
main:
    printf "Hello, World!"
    li  a7, 93
    ecall
"""

# Create a source for our assembly code
source = RiscemuSource("example.asm", StringIO(code))
# Register the source with riscemu
main.input_files.append(source)

try:
    # Execute our code
    main.run()
except Exception as ex:
    # Print a pretty string representation in case of an error
    print(ex)

Hello, World!


In the next post, we'll look at running numeric computations.

I am [Sasha Lopoukhine](lopoukhine.com), a PhD student at the University of Cambridge.
You can contact me with your feedback at sasha@lopoukhine.com.